In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torchaudio
import kaldiio
import torch
import torch.nn as nn

import random
from torch.nn.utils.rnn import pack_padded_sequence
from torch import nn, optim

In [3]:
# read utt2dur file for "BN"
file_path = '/mnt/hdd1/MAD_ASR_hdd1/mad_trail/'

file = '/mnt/hdd1/MAD_ASR_hdd1/mad_trail/bn/train/D5/feats.pt'


/mnt/hdd1/MAD_ASR_hdd1/mad_trail/bn/train/D5/feats.pt


In [7]:
# data = kaldiio.load_ark(file)

data = torch.load(file)


In [15]:
# Convert data to tensors
tensor_dict = {}
feats = []
# Convert data to tensors and store in the dictionary
for key, value in data:
    # tensor_dict[key] = torch.tensor(value).unsqueeze(0)
    # print(torch.tensor(value).shape)
    # print((torch.tensor(value).clone().detach()).shape)
    feats.append(torch.tensor(value).clone().detach())



/tmp/ipykernel_6972/3176914319.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  feats.append(torch.tensor(value).clone().detach())


In [16]:
print(len(feats))

80


In [17]:
# collate 
def collate_fn(batch):
    # Get the maximum sequence length along the third dimension
    max_length = max(sample.shape[1] for sample in batch)
    # Pad the samples to have the same length
    padded_batch = []
    for sample in batch:
        # print(sample.shape)
        pad_length = max_length - sample.shape[1]
        padded_sample = torch.nn.functional.pad(sample, (0, pad_length))
        # print(padded_sample.shape)
        padded_batch.append(padded_sample)

    return torch.stack(padded_batch)


In [18]:
# dataloader

from torch.utils.data import DataLoader 

dataloader = DataLoader(feats, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [19]:
# check = iter(dataloader)
# for i in check:
#     print(type(i))
#     print(i.shape)
#     break

<class 'torch.Tensor'>
torch.Size([32, 512, 604])


In [193]:
###########   Task 1 ---- invert input (x, 512) -- done

##########    Task 2 ---- check LSTM -- Bi LSTM

In [20]:
### Lstm blocks
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):

        # out,_ = self.lstm(x, (h0, c0))
        x=x.transpose(1,2)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Use only the last output of the sequence
        # out = out.transpose(1, 2).reshape(-1, batch_size).transpose(1, 0)
        return out
    

In [21]:
### Bi-Lstm blocks
class BILSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(BILSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # out,_ = self.lstm(x, (h0, c0))
        x=x.transpose(1,2)
        out, _ = self.lstm(x)
        
        out = self.linear(out[:, -1, :])  # Use only the last output of the sequence
        return out
 
   

In [22]:
it = iter(dataloader)
first = next(it)
input_size=first.shape[1]
hidden_size = 128
num_layers = 4
output_size = 8  # Modify this according to your specific task

lstm_model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# lstm_model = LSTMModel(input_size, hidden_size, num_layers, output_size)

In [25]:
for inputs in dataloader:
    print('shape of input',inputs.shape)
    output=lstm_model(inputs)
    print('shape of output', output.shape)


shape of input torch.Size([32, 512, 604])
shape of output torch.Size([32, 8])
shape of input torch.Size([32, 512, 472])
shape of output torch.Size([32, 8])
shape of input torch.Size([16, 512, 504])
shape of output torch.Size([16, 8])


In [24]:
print(output)

tensor([[-0.0387, -0.0555, -0.0513, -0.0080,  0.0287,  0.0031, -0.0486, -0.0581],
        [-0.0387, -0.0555, -0.0513, -0.0080,  0.0287,  0.0031, -0.0486, -0.0581],
        [-0.0387, -0.0555, -0.0514, -0.0081,  0.0287,  0.0033, -0.0485, -0.0581],
        [-0.0387, -0.0555, -0.0513, -0.0080,  0.0287,  0.0031, -0.0486, -0.0581],
        [-0.0387, -0.0555, -0.0513, -0.0080,  0.0287,  0.0031, -0.0486, -0.0581],
        [-0.0387, -0.0555, -0.0513, -0.0080,  0.0287,  0.0031, -0.0486, -0.0581],
        [-0.0387, -0.0555, -0.0513, -0.0080,  0.0287,  0.0031, -0.0486, -0.0581],
        [-0.0387, -0.0555, -0.0513, -0.0080,  0.0287,  0.0031, -0.0486, -0.0581],
        [-0.0387, -0.0555, -0.0513, -0.0080,  0.0287,  0.0031, -0.0486, -0.0581],
        [-0.0387, -0.0555, -0.0513, -0.0080,  0.0287,  0.0031, -0.0486, -0.0581],
        [-0.0387, -0.0555, -0.0513, -0.0080,  0.0287,  0.0031, -0.0486, -0.0581],
        [-0.0387, -0.0555, -0.0513, -0.0080,  0.0287,  0.0031, -0.0486, -0.0581],
        [-0.0387

In [ ]:
def train(model,trainloader, valloader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # model.train().to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    val_loss_min = np.inf

    train_losses = []
    val_losses = []

    train_accuracies = []
    val_accuracies = []

    for i_epoch in range(n_epochs):
        epoch_train_loss = 0
        epoch_train_acc = 0
        model.train().to(device)
        for batch in tqdm(trainloader, desc="Train"):
            batch_x, batch_y = batch
            # batch_x, batch_y = batch_x.float().to(device), batch_y.long().view(-1).to(device)
            y_hat = model(batch_x)
            loss = criterion(y_hat, batch_y)
            epoch_train_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, max_indices = torch.max(y_hat, 1)
            epoch_train_acc += (max_indices == batch_y).sum().data.cpu().numpy()/max_indices.size()[0]
    
        epoch_train_loss = epoch_train_loss/len(trainloader)
        epoch_train_acc = epoch_train_acc/len(trainloader)
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)


        model.eval()
        with torch.no_grad():
            epoch_val_loss = 0
            epoch_val_acc = 0
            for batch in tqdm(valloader, desc="Val  "):
                batch_x, batch_y = batch
                batch_x, batch_y = batch_x.float().to(device), batch_y.long().view(-1).to(device)
                y_hat = model(batch_x)
                loss = criterion(y_hat, batch_y)
                epoch_val_loss += loss.item()

                _, max_indices = torch.max(y_hat, 1)
                epoch_val_acc += (max_indices == batch_y).sum().data.cpu().numpy()/max_indices.size()[0]

            epoch_val_loss = epoch_val_loss/len(valloader)
            epoch_val_acc = epoch_val_acc/len(valloader)
            val_losses.append(epoch_val_loss)
            val_accuracies.append(epoch_val_acc)

            if epoch_val_loss < val_loss_min:
                torch.save(model.state_dict(), save_path)
                tqdm.write(f'Validation loss reduced from {val_loss_min:.6f} to {epoch_val_loss:.6f}, saving model at {save_path} ...')
                val_loss_min = epoch_val_loss

        if scheduler:
            scheduler.step()
            tqdm.write(f'Updating lr to {scheduler.get_last_lr()}')

        tqdm.write(f'Epoch : {i_epoch+1:02}\nTrain Loss = {epoch_train_loss:.4f}\tTrain Acc = {epoch_train_acc}\n  Val Loss = {epoch_val_loss:.4f}\t  Val Acc = {epoch_val_acc}\n')

    loss_dict = {"train_losses" : train_losses,
                 "val_losses" : val_losses,
                 "train_accuracies" : train_accuracies,
                 "val_Accuracies" : val_accuracies}

    return loss_dict

